In [ ]:
import numpy as np
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis
from portfolio_hubo_qaoa import HigherOrderPortfolioQAOA
import random
random.seed(5)

n_stocks = 3
stocks = ["AAPL", "GOOGL", "MSFT", "AMZN", "META", "TSLA", "WMT", "CAT", "KO", "IBM", "INTC", "CSCO", "ORCL", "QCOM", 
          "NVDA", "ADBE", "PYPL", "CRM", "ACN", "TXN", "AVGO", "INTU", "AMAT", "MU"]
stocks = random.sample(stocks, n_stocks)

data = yf.download(stocks, start="2021-01-01", end="2024-01-01")
prices_now = data["Close"].iloc[-1]
print(prices_now)
returns = data["Close"].pct_change().dropna()
stocks = returns.columns

numpy_returns = returns.to_numpy()
expected_returns = numpy_returns.mean(axis=0)*252
print(returns.mean())
print(expected_returns)
covariance_matrix = np.cov(numpy_returns, rowvar=False)*252
coskewness_tensor = coskewness(numpy_returns)#*(252**2)
cokurtosis_tensor = cokurtosis(numpy_returns)#*(252**3)

# Budget in dollars
# We compute the maximum number of shares we can buy for each stock
# At most n_qubits for each stock
n_qubits = 4
cheapest_stock = prices_now.min()
budget = n_qubits*cheapest_stock
print(budget)

risk_aversion = 3

portfolio_hubo = HigherOrderPortfolioQAOA(stocks=stocks,
                                          prices_now=prices_now,
                                          expected_returns=expected_returns, 
                                          covariance_matrix=covariance_matrix,
                                          budget=budget,
                                          coskewness_tensor=coskewness_tensor, 
                                          cokurtosis_tensor=cokurtosis_tensor,
                                          log_encoding = True, 
                                          layers = 1,
                                          risk_aversion = risk_aversion)

smallest_eigenvalues, smallest_bitstrings, first_excited_energy, optimized_portfolio, second_optimized_portfolio = portfolio_hubo.solve_exactly()

In [ ]:
print(smallest_eigenvalues)
print(smallest_bitstrings)
print(first_excited_energy)
print("Best: ", optimized_portfolio)
print("Second best: ", second_optimized_portfolio)
print("objective value 1: ", portfolio_hubo.get_objective_value(stocks, optimized_portfolio[0]))
print("objective value 2: ", portfolio_hubo.get_objective_value(stocks, second_optimized_portfolio[0]))

[np.float64(-17657567.405618757)]
[[1, 0, 0, 1, 0]]
-17641740.71959126
Best:  [{'KO': 1, 'MU': 2, 'TXN': 0}]
Second best:  [{'KO': 1, 'MU': 0, 'TXN': 1}]
objective value 1:  -12.046821326974943
objective value 2:  -1.689788650703018


In [ ]:
portfolio_hubo.solve_with_qaoa_jax()

Most probable state: 01111 and [[1, 0, 0, 1, 0]] with probs 01111
Most probable state: 11010 and [[1, 0, 0, 1, 0]] with probs 11010
Most probable state: 10010 and [[1, 0, 0, 1, 0]] with probs 10010
Two most probable states: ['11111', '10010'] with probabilities [Array(0.03208242, dtype=float32), Array(0.0324635, dtype=float32)]
Optimized portfolios: [{'KO': 3, 'MU': 3, 'TXN': 1}, {'KO': 1, 'MU': 2, 'TXN': 0}]
Budget constraint not satisfied 591.0028343200684 228.7340850830078 Difference:  362.26874923706055
Budget constraint satisfied 227.13750076293945 228.7340850830078 Difference:  1.5965843200683594
Final expectation value: -9419828.0
Objective values: [np.float64(-427.89801772090607), np.float64(-12.046821326974943)]


(['11111', '10010'],
 Array(-9419828., dtype=float32),
 Array([[-0.02238139],
        [-0.00596312]], dtype=float32),
 30,
 [Array(0.03208242, dtype=float32), Array(0.0324635, dtype=float32)],
 [{'KO': 3, 'MU': 3, 'TXN': 1}, {'KO': 1, 'MU': 2, 'TXN': 0}])

In [ ]:
#two_most_probable_states, smallest_eigenvalue, params, total_steps, states_probs, optimized_portfolios = portfolio_hubo.solve_with_iterative_QAOA(max_layers=1)

In [ ]:
from portfolio_higher_moments_classical import HigherMomentPortfolioOptimizer


hef = HigherMomentPortfolioOptimizer(stocks= stocks,
                                     expected_returns=expected_returns, 
                                     covariance_matrix=covariance_matrix, 
                                     coskewness=coskewness_tensor, 
                                     cokurtosis=cokurtosis_tensor, 
                                     risk_aversion=risk_aversion)

weights = hef.optimize_portfolio_with_higher_moments()
print("Optimized weights:")
for asset, weight in zip(stocks, weights):
    print(f"{asset}: {weight:.2%}")
    
allocation, left_overs = hef.get_discrete_allocation(prices_now, budget)
print("Left over budget: ", left_overs)
print("Ratio of left over budget: ", left_overs/budget)

print("Optimized Discrete Allocation:")
for asset, amount in allocation.items():
    print(f"{asset}: {amount}")
    
value = portfolio_hubo.get_objective_value(stocks, allocation)
print("objective value: ", value)

Optimized weights:
KO: 29.56%
MU: 33.01%
TXN: 37.43%
Left over budget:  29.390052771718985
Ratio of left over budget:  0.12849004450322
Optimized Discrete Allocation:
KO: 2
MU: 1
objective value:  -1.3393356320931435
